In [40]:
import pandas as pd
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
#import geopandas as gpd
import geopy as gp
from geopy.geocoders import GoogleV3
from tqdm import tqdm

In [3]:
df = pd.read_csv('all_countries_data.csv')


In [5]:
df.Country.unique()

array(['no data1', 'no data2', 'New Zealand', 'no data4', 'no data5',
       'no data6', 'Somalia', 'China', 'Japan', 'United States',
       'South Korea', 'Spain', 'Canada', 'Italy', 'Slovenia', 'Germany',
       'Netherlands', 'Finland', 'no data101', 'no data102'], dtype=object)

In [61]:
japan_df = df[df['Country'] == "Japan"]

In [7]:
japan_df

,Unnamed: 0,Captured Time,Latitude,Longitude,Value,Height,Country
295606,12,2022-03-19 01:58:23,31.833193,130.301922,15.0,13.0,Japan
295607,13,2022-03-19 01:58:23,31.833193,130.301922,30.0,13.0,Japan
295608,55,2022-03-19 01:53:20,31.833193,130.301922,15.0,13.0,Japan
295609,56,2022-03-19 01:53:20,31.833193,130.301922,29.0,13.0,Japan
295610,98,2022-03-19 01:48:17,31.833193,130.301922,14.0,13.0,Japan
...,...,...,...,...,...,...,...
26828740,31278737,2017-06-19 07:11:21,37.796306,140.514413,16.0,65.0,Japan
26828741,31278738,2017-06-19 07:11:14,37.796306,140.514413,39.0,65.0,Japan
26828742,31278783,2017-06-19 07:06:02,37.796306,140.514413,14.0,65.0,Japan
26828743,31278784,2017-06-19 07:05:55,37.796306,140.514413,35.0,65.0,Japan


In [21]:
japan_grouped = japan_df.groupby(['Latitude', 'Longitude'], as_index = False).count()

In [22]:
japan_grouped.sort_values(by = ['Latitude', 'Longitude'], inplace = True)

In [23]:
japan_grouped.columns

Index(['Latitude', 'Longitude', 'Unnamed: 0', 'Captured Time', 'Value',
       'Height', 'Country'],
      dtype='object')

In [24]:
japan_cords = japan_grouped[['Latitude', 'Longitude']]


In [25]:
japan_cords

,Latitude,Longitude
0,31.833193,130.301922
1,32.475844,130.606620
2,32.475932,130.607123
3,33.546837,130.179520
4,33.546837,130.179535
...,...,...
5259,37.787500,140.552400
5260,37.792805,140.930957
5261,37.794237,140.439040
5262,37.796306,140.514413


In [27]:
japan_cords = japan_cords.values.tolist()

In [59]:
def get_regions(cord_list):
    locator = Nominatim(user_agent="ekspllo_new1", timeout=10)
    city_dict = {}
    for i in tqdm(range(len(cord_list))):
        pos = str(cord_list[i][0]) + ', ' + str(cord_list[i][1])
        location = locator.reverse(pos, timeout = 10, language="en")
        try:
          city =  location.raw['address']['province']
        except:
          city = "no data" 
        city_dict[pos] = city
    return city_dict

In [60]:
regions = get_regions(japan_cords)

100%|██████████████████████████████████████████████████████████████████████████████| 5264/5264 [43:52<00:00,  2.00it/s]


In [64]:
regions

{'31.833193, 130.301922': 'Kagoshima Prefecture',
 '32.475844, 130.60662': 'Kumamoto Prefecture',
 '32.475932, 130.607123': 'Kumamoto Prefecture',
 '33.546837, 130.17952': 'Fukuoka Prefecture',
 '33.546837, 130.179535': 'Fukuoka Prefecture',
 '34.476948, 136.161301': 'Nara Prefecture',
 '34.478188, 136.164352': 'Nara Prefecture',
 '34.478329, 136.165024': 'Nara Prefecture',
 '34.478954, 136.164825': 'Nara Prefecture',
 '34.479305, 136.16095': 'Nara Prefecture',
 '34.479832, 136.160614': 'Nara Prefecture',
 '34.480247, 136.161682': 'Nara Prefecture',
 '34.480267, 136.161652': 'Nara Prefecture',
 '34.480274, 136.161652': 'Nara Prefecture',
 '34.480328, 136.163727': 'Nara Prefecture',
 '34.480492, 136.16069': 'Nara Prefecture',
 '34.48053, 136.163559': 'Nara Prefecture',
 '34.480568, 136.161346': 'Nara Prefecture',
 '34.480572, 136.161301': 'Nara Prefecture',
 '34.480595, 136.162354': 'Nara Prefecture',
 '34.480625, 136.164322': 'Nara Prefecture',
 '34.480751, 136.162277': 'Nara Prefectur

In [65]:
print(set(regions.values()))

{'Kanagawa Prefecture', 'Fukuoka Prefecture', 'Miyagi Prefecture', 'Fukushima Prefecture', 'Shiga Prefecture', 'Chiba Prefecture', 'Kumamoto Prefecture', 'Kagoshima Prefecture', 'Mie Prefecture', 'no data', 'Saitama Prefecture', 'Ibaraki Prefecture', 'Nara Prefecture'}


In [66]:
japan_df['Province'] = japan_df.apply(lambda row: regions.get(str(row['Latitude']) + ', ' + str(row['Longitude'])), axis=1)

In [67]:
japan_df

,Unnamed: 0,Captured Time,Latitude,Longitude,Value,Height,Country,Province
295606,12,2022-03-19 01:58:23,31.833193,130.301922,15.0,13.0,Japan,Kagoshima Prefecture
295607,13,2022-03-19 01:58:23,31.833193,130.301922,30.0,13.0,Japan,Kagoshima Prefecture
295608,55,2022-03-19 01:53:20,31.833193,130.301922,15.0,13.0,Japan,Kagoshima Prefecture
295609,56,2022-03-19 01:53:20,31.833193,130.301922,29.0,13.0,Japan,Kagoshima Prefecture
295610,98,2022-03-19 01:48:17,31.833193,130.301922,14.0,13.0,Japan,Kagoshima Prefecture
...,...,...,...,...,...,...,...,...
26828740,31278737,2017-06-19 07:11:21,37.796306,140.514413,16.0,65.0,Japan,Fukushima Prefecture
26828741,31278738,2017-06-19 07:11:14,37.796306,140.514413,39.0,65.0,Japan,Fukushima Prefecture
26828742,31278783,2017-06-19 07:06:02,37.796306,140.514413,14.0,65.0,Japan,Fukushima Prefecture
26828743,31278784,2017-06-19 07:05:55,37.796306,140.514413,35.0,65.0,Japan,Fukushima Prefecture


In [70]:
japan_df.to_csv('japan_df_province')